# Using DuckDB's SQL as an alternative workflow for large CSV / Parquet files

For large CSV files, we may look to use Dask or PySpark even, when loading the CSV to a database is not possible.  But, there is an alternative SQL-based workflow using DuckDB.

### Let's start with Dask workflow

In [ ]:
import dask.dataframe as dd
import numpy as np

#### For best practice, it is usually best to define data type, instead of having dask infer/guess the data type (HINT: type inference is often wrong and leads to poor performance)

In [ ]:
col_types = {
    'SUPP_NO': str,
    'ASN_NO': str,
    'ACTUAL_DT': str,
    'ACTUAL_TIME': str,
    'DC_SEG': 'category',
    'TRLR_NO': str,
    'TRLR_ARR_DT': str,
    'TRLR_ARR_TIME': str,
    'ORDERED_QTY': np.int32,   # Use lowest, memory-consuming "bit-ness" (Are we going to order more than 64K parts?  Negative values?)
    'ASN_PART_QTY': np.int32,         # ditto
    'PART_UNLD_QTY': np.int32,        # ditto
    'TOTAL_ASN_PART_QTY': np.int32,   # ditto
}

In [ ]:
df = dd.read_csv('shipping_receiving_all_plants.csv', dtype=col_types)

In [ ]:
df.head(2).transpose()

#### Let's time how long it would take to sum a column using the csv file

In [ ]:
%%timeit
df['ORDERED_QTY'].sum().compute()

#### Thanks to Dask's parallelization, we didn't blow out our local machine's memory, but it still took on average almost 50 seconds.  Can we do better?

## Querying the parquet format using Dask instead

As before, we should define the data type or schema for the columns.  Parquet's underlying technology is Apache Arrow, so we need to use pyarrow's data types instead.

In [ ]:
import pyarrow as pa

#### Converting CSV to parquet format

In [ ]:
col_schema = {
    'SUPP_NO': pa.string(),
    'ASN_NO': pa.string(),
    'ACTUAL_DT': pa.string(),
    'ACTUAL_TIME': pa.string(),
    'DC_SEG': pa.string(),
    'TRLR_NO': pa.string(),
    'TRLR_ARR_DT': pa.string(),
    'TRLR_ARR_TIME': pa.string(),
    'ORDERED_QTY': pa.int32(),
    'ASN_PART_QTY': pa.int32(),
    'PART_UNLD_QTY': pa.int32(),
    'TOTAL_ASN_PART_QTY': pa.int32(),
}

#### Now save Dask dataframe as a parquet file.  UPDATE: Later found out that repartioning to a single partition, boosted performance.

In [ ]:
%%time
df = df.repartition(npartitions=1)
df.to_parquet('s_r_all_plants.parquet', write_index=False, schema=col_schema)

The conversion resulted in a folder called `s_r_all_plants.parquet` folder containing a single partition parquet file.

In [ ]:
!dir s_r_all_plants.parquet

With Dask, we don't need to read all the individual parquet files, just reference the folder containing them:

In [ ]:
%%time
dfp = dd.read_parquet('s_r_all_plants.parquet/')

In [ ]:
dfp.head()

#### Computing the sum on parquet file took less than a second

In [ ]:
%%timeit
dfp['ORDERED_QTY'].sum().compute()

But, what if you don't want to use dataframe API?  But a SQL workflow instead?

## SQL Workflow using DuckDB

Starting with version 0.2.7, DuckDB now supports querying parquet files!  Just `pip install duckdb`, then you're good to go.

In [ ]:
import duckdb

In [ ]:
duckdb.__version__

#### I prefer to use context manager (using `with` statement) when connecting to databases.  But, DuckDB does not yet support context management.  So I have to implement a custom class myself.

In [ ]:
class DuckDBConn:
    """"""
    def __init__(self):
        """Constructor"""
        pass
    def __enter__(self):
        """
        Open the database connection
        """
        self.conn = duckdb.connect()
        return self.conn
    def __exit__(self, exc_type, exc_val, exc_tb):
        """
        Close the connection
        """
        self.conn.close()
        if exc_val:
            raise

DuckDB supports wildcard "globbing" with the asterisk symbol to read in multiple parquet files:

In [ ]:
%%timeit
with DuckDBConn() as con:
    df_sr = con.execute("""
        SELECT sum(ORDERED_QTY) as sum_ordered
        FROM 's_r_all_plants.parquet/*.parquet'
    """).df()
    
df_sr

DuckDB was just as fast as Dask.

Let's try to convert the parquet files into a single parquet file to see if there could be performance improvements.  But the problem is, there doesn't seem to be good documentation to determine optimal `row_group_size`.  Through trial and error, I settled on 200K rows.

In [ ]:
import pyarrow.parquet as pq

In [ ]:
pq.write_table(pq.ParquetDataset('s_r_all_plants.parquet/').read(), 'shipping_receiving.parquet')

In [ ]:
!dir

In [ ]:
%%timeit
with DuckDBConn() as con:
    df_sr = con.execute("""
       SELECT sum(ORDERED_QTY) as sum_ordered
       FROM 'shipping_receiving.parquet'
       """).df()
    
df_sr

With DuckDB, we got some speed boost with a single file, parquet file.

#### Let's see if there is a difference with Dask

In [ ]:
dfp = dd.read_parquet('shipping_receiving.parquet')

In [ ]:
%%time
dfp['ORDERED_QTY'].sum().compute()

With Dask reading a single file, parquet file, it did not get much, if any speed boost.